# Simulation Studies of Linear QR Models

In [1]:
import time
import pandas as pd
import numpy as np
import numpy.random as rgt
from scipy.stats import norm, t
from joblib import Parallel, delayed

from quantes.linear import low_dim
from utl import cov_generate

# Number of monte carlo simulations
M = 500 

## Homoscedastic model

### Estimation error

In [2]:
n, p = 8000, 400
itcp, beta = 4, 1*np.ones(p)*(2*rgt.binomial(1, 1/2, p) - 1)
tau, t_df = 0.75, 2

def sim(m):
    rgt.seed(m)
    X = rgt.normal(0, 1.5, size=(n,p))
    Y = itcp + X @ beta + rgt.standard_t(t_df, n) - t.ppf(tau, t_df)
    tic = time.time()
    model = low_dim(X, Y).fit(tau=tau)
    runtime = time.time() - tic
    itcp_err = (model['beta'][0] - itcp)**2
    coef_err = np.sum((model['beta'][1:] - beta)**2)
    return itcp_err, coef_err, runtime

results = Parallel(n_jobs=-1)(delayed(sim)(m) for m in range(M)) 
itcp_se, coef_se, runtime = zip(*results)
out = pd.DataFrame({'MSE (itcp)': np.mean(itcp_se),
                    'std (itcp)': np.std(itcp_se),
                    'MSE (coef)': np.mean(coef_se),
                    'std (coef)': np.std(coef_se),
                    'Runtime': np.mean(runtime)}, index=['conquer'])
out

,MSE (itcp),std (itcp),MSE (coef),std (coef),Runtime
conquer,0.001963,0.001914,0.076628,0.005824,0.260992


### Coverage of confidence intervals

In [3]:
n, p = 500, 20
mask = 2*rgt.binomial(1, 1/2, p) - 1
itcp, beta = 4, 1*np.ones(p)*mask
tau, t_df = 0.75, 2

def homo_sim(m=0):
    rgt.seed(42+m)
    covers, widths = np.zeros((4, p)), np.zeros((4, p))
    X = rgt.normal(0, 1.5, size=(n,p))
    Y = itcp + X@beta + rgt.standard_t(t_df, n) - t.ppf(tau, t_df)
    model = low_dim(X, Y)    
    sol1 = model.norm_ci(tau, solver='BBGD')
    sol2 = model.mb_ci(tau, solver='BBGD')
    covers[0,:] = (beta >= sol1['normal'][1:,0])*(beta<= sol1['normal'][1:,1])
    covers[1,:] = (beta >= sol2['percentile'][1:,0])*(beta<= sol2['percentile'][1:,1])
    covers[2,:] = (beta >= sol2['pivotal'][1:,0])*(beta<= sol2['pivotal'][1:,1])
    covers[3,:] = (beta >= sol2['normal'][1:,0])*(beta<= sol2['normal'][1:,1])
    widths[0,:] = sol1['normal'][1:,1] - sol1['normal'][1:,0]
    widths[1,:] = sol2['percentile'][1:,1] - sol2['percentile'][1:,0]
    widths[2,:] = sol2['pivotal'][1:,1] - sol2['pivotal'][1:,0]
    widths[3,:] = sol2['normal'][1:,1] - sol2['normal'][1:,0]
    return covers, widths

results = Parallel(n_jobs=-1)(delayed(homo_sim)(m) for m in range(M))
ci_cover, ci_width = zip(*results)
cover = pd.DataFrame(np.sum(ci_cover, axis=0) / M, 
                     index=["Normal", "MB-Perc", "MB-Piv", "MB-Norm"])
cover.columns = pd.Index(np.linspace(1, p, p), dtype=int)
print(cover)
print()
width = pd.DataFrame(np.mean(ci_width, axis=0), 
                     index=["Normal", "MB-Perc", "MB-Piv", "MB-Norm"]).round(3)
width.columns = cover.columns
print(width)

            1      2      3      4      5      6      7      8      9      10  \
Normal   0.966  0.970  0.978  0.960  0.972  0.968  0.974  0.962  0.964  0.960   
MB-Perc  0.952  0.970  0.960  0.970  0.966  0.950  0.970  0.964  0.958  0.966   
MB-Piv   0.932  0.960  0.932  0.932  0.932  0.928  0.940  0.922  0.932  0.922   
MB-Norm  0.960  0.978  0.958  0.968  0.966  0.950  0.966  0.950  0.956  0.944   

            11     12     13     14     15     16     17     18     19     20  
Normal   0.964  0.960  0.960  0.976  0.960  0.986  0.968  0.970  0.966  0.948  
MB-Perc  0.964  0.954  0.960  0.986  0.962  0.978  0.966  0.956  0.974  0.958  
MB-Piv   0.948  0.908  0.924  0.934  0.910  0.952  0.942  0.936  0.918  0.918  
MB-Norm  0.976  0.942  0.952  0.972  0.954  0.984  0.972  0.958  0.956  0.952  

            1      2      3      4      5      6      7      8      9      10  \
Normal   0.255  0.260  0.259  0.259  0.264  0.259  0.261  0.263  0.260  0.261   
MB-Perc  0.222  0.226  0.222  0

## Heteroscedastic model

Let $z=(z_1, \ldots, z_p)^T \sim N(0, \Sigma)$ with $\Sigma = (0.5^{|j-k|})_{1\leq j, k \leq p}$ and $z_0 \sim {\rm Unif}(0,2)$ be independent. Generate independent data vectors $\{(y_i , x_i) \}_{i=1}^n$ from the model 
$$
    y_i =  \varepsilon_i x_{i1}  +  x_{i2} + \cdots + x_{ip}   \quad {\rm with } \ \  x_i = (x_{i1}, \ldots, x_{ip})^T \sim (z_0, z_2, \ldots, z_p)^T,
$$
where $\varepsilon_i$'s are iid $N(0,1)$ variables that are independent of $x_i$'s.

Consider two quantile levels: $\tau=0.5$ and $\tau=0.8$. Note that the effect of $x_{i1}$ is only present for $\tau=0.8$.

In [4]:
n, p = 2000, 10
beta = np.ones(p)
beta[0] = 0
mu, Sig = np.zeros(p), cov_generate(np.ones(p), 0.5)

def hetero_sim(m=0, tau=0.5, true_beta=beta):
    rgt.seed(42+m)
    X = rgt.multivariate_normal(mean=mu, cov=Sig, size=n)
    X[:,0] = rgt.uniform(0, 2, size=n)
    Y = X@beta +  X[:,0]*rgt.normal(0,1,size=n)
    covers, widths = np.zeros([4, p]), np.zeros([4, p])
    model = low_dim(X, Y, intercept=False)
    sol1 = model.norm_ci(tau)
    sol2 = model.mb_ci(tau)
    covers[0,:] = (true_beta >= sol1['normal'][:,0])*(true_beta<= sol1['normal'][:,1])
    covers[1,:] = (true_beta >= sol2['percentile'][:,0])*(true_beta<= sol2['percentile'][:,1])
    covers[2,:] = (true_beta >= sol2['pivotal'][:,0])*(true_beta<= sol2['pivotal'][:,1])
    covers[3,:] = (true_beta >= sol2['normal'][:,0])*(true_beta<= sol2['normal'][:,1])
    widths[0,:] = sol1['normal'][:,1] - sol1['normal'][:,0]
    widths[1,:] = sol2['percentile'][:,1] - sol2['percentile'][:,0]
    widths[2,:] = sol2['pivotal'][:,1] - sol2['pivotal'][:,0]
    widths[3,:] = sol2['normal'][:,1] - sol2['normal'][:,0]
    return covers, widths

### Case 1: $\tau=0.5$.
The conditional median of $y_i$ given $x_i$ is $Q_{0.5}(y_i | x_i) =  x_{i2} + \cdots + x_{ip}$.

In [5]:
tau = 0.5
true_beta = np.copy(beta)
true_beta[0] = norm.ppf(tau)

results = Parallel(n_jobs=-1)(delayed(hetero_sim)(m, tau, true_beta) 
                              for m in range(M))
ci_cover, ci_width = zip(*results)
cover = pd.DataFrame(np.sum(ci_cover, axis=0) / M, 
                     index=["Normal", "MB-Perc", "MB-Piv", "MB-Norm"])
cover.columns = pd.Index(np.linspace(1, p, p), dtype=int)
print(cover)
print()
width = pd.DataFrame(np.mean(ci_width, axis=0), 
                     index=["Normal", "MB-Perc", "MB-Piv", "MB-Norm"]).round(3)
width.columns = cover.columns
print(width)

            1      2      3      4      5      6      7      8      9      10
Normal   0.946  0.946  0.956  0.964  0.956  0.940  0.948  0.966  0.962  0.952
MB-Perc  0.936  0.948  0.956  0.956  0.944  0.936  0.944  0.952  0.944  0.940
MB-Piv   0.934  0.960  0.960  0.974  0.966  0.954  0.954  0.972  0.958  0.964
MB-Norm  0.944  0.966  0.964  0.974  0.962  0.952  0.954  0.968  0.962  0.952

            1      2      3      4      5      6      7      8      9      10
Normal   0.125  0.063  0.071  0.070  0.071  0.071  0.070  0.070  0.070  0.063
MB-Perc  0.121  0.065  0.073  0.073  0.073  0.073  0.073  0.072  0.072  0.065
MB-Piv   0.121  0.065  0.073  0.073  0.073  0.073  0.073  0.072  0.072  0.065
MB-Norm  0.124  0.066  0.074  0.074  0.074  0.074  0.074  0.073  0.074  0.066


### Case 2: $\tau=0.8$. 
In this case, the conditional $0.8$-quantile of $y_i$ given $x_i$ is $Q_{0.8}(y_i | x_i) =   \Phi^{-1}(0.8) x_{i1} + x_{i2} + \cdots + x_{ip}$.

In [6]:
tau = 0.8
true_beta = np.copy(beta)
true_beta[0] = norm.ppf(tau)

results = Parallel(n_jobs=-1)(delayed(hetero_sim)(m, tau, true_beta) for m in range(M))
ci_cover, ci_width = zip(*results)
cover = pd.DataFrame(np.sum(ci_cover, axis=0) / M, 
                     index=["Normal", "MB-Perc", "MB-Piv", "MB-Norm"])
cover.columns = pd.Index(np.linspace(1, p, p), dtype=int)
print(cover)
print()
width = pd.DataFrame(np.mean(ci_width, axis=0), 
                     index=["Normal", "MB-Perc", "MB-Piv", "MB-Norm"]).round(3)
width.columns = cover.columns
print(width)

            1      2      3      4      5      6      7      8      9      10
Normal   0.948  0.958  0.954  0.970  0.958  0.962  0.942  0.952  0.964  0.952
MB-Perc  0.942  0.952  0.938  0.952  0.950  0.936  0.942  0.946  0.952  0.922
MB-Piv   0.934  0.962  0.958  0.978  0.970  0.966  0.958  0.960  0.978  0.966
MB-Norm  0.938  0.964  0.958  0.972  0.966  0.960  0.950  0.958  0.970  0.968

            1      2      3      4      5      6      7      8      9      10
Normal   0.143  0.065  0.072  0.072  0.073  0.073  0.072  0.072  0.072  0.065
MB-Perc  0.139  0.067  0.075  0.075  0.076  0.076  0.075  0.075  0.075  0.068
MB-Piv   0.139  0.067  0.075  0.075  0.076  0.076  0.075  0.075  0.075  0.068
MB-Norm  0.141  0.068  0.076  0.076  0.077  0.077  0.076  0.076  0.077  0.069
